In [1]:
import snap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import logging

# enable logging
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug('hello')

DEBUG:root:hello


## 1 Network Characteristics

### 1.1 Degree Distribution

In [ ]:
# %%time
# Problem 1.1
def genErdosRenyi(N=5242, E=14484):
    """
    :param - N: number of nodes
    :param - E: number of edges

    return type: snap.PUNGraph
    return: Erdos-Renyi graph with N nodes and E edges
    """
    ############################################################################
    # TODO: Your code here!

    G = snap.TUNGraph.New()
    for idx in range(N):
        G.AddNode(idx)
        
    while (G.GetEdges() < E):
        edges = np.random.choice(range(5242),2,replace=False)
        G.AddEdge(int(edges[0]),int(edges[1]))
    ############################################################################
    return G

random_graph = genErdosRenyi()
assert random_graph.GetNodes() == 5242
# print('edges : ',  G.GetEdges())
assert random_graph.GetEdges() == 14484

In [ ]:
# config
N = 5242
# def add_edge(G, node_id, src, dst):
#     G.
def genCircle(N=5242):
    """
    :param - N: number of nodes

    return type: snap.PUNGraph
    return: Circle graph with N nodes and N edges. Imagine the nodes form a
        circle and each node is connected to its two direct neighbors.
    """
    ############################################################################
    G = snap.TUNGraph.New()
    for idx in range(N):
        G.AddNode(idx)
        
    for idx in range(N):
        if idx == (N-1):
            G.AddEdge(N-1,0)
        else:
            G.AddEdge(idx,idx+1)
    ############################################################################
    return G

# testing it
G = genCircle()
assert G.GetNodes() == 5242
assert G.GetEdges() == 5242


def connectNbrOfNbr(Graph, N=5242):
    """
    :param - Graph: snap.PUNGraph object representing a circle graph on N nodes
    :param - N: number of nodes

    return type: snap.PUNGraph
    return: Graph object with additional N edges added by connecting each node
        to the neighbors of its neighbors
    """
    ############################################################################
    for idx in range(N):
        if idx == (N-1):
            Graph.AddEdge(idx, 1)
        elif idx == (N-2):
            Graph.AddEdge(idx, 0)
        else:
            Graph.AddEdge(idx, idx+2)

    ############################################################################
    return Graph

G = connectNbrOfNbr(G)
assert G.GetNodes() == N
assert G.GetEdges() == N * 2

def connectRandomNodes(Graph, M=4000):
    """
    :param - Graph: snap.PUNGraph object representing an undirected graph
    :param - M: number of edges to be added

    return type: snap.PUNGraph
    return: Graph object with additional M edges added by connecting M randomly
        selected pairs of nodes not already connected.
    """
    ############################################################################
    num_edges = Graph.GetEdges()
    num_nodes = Graph.GetNodes()
    while Graph.GetEdges() < (num_edges + M):
        edge = np.random.choice(range(num_nodes),2,replace=False)
        Graph.AddEdge(int(edge[0]), int(edge[1]))
    ############################################################################
    return Graph

G = connectRandomNodes(G)
print('num edges: ', G.GetEdges() )
assert G.GetEdges() == 2*N + 4000

In [ ]:
def genSmallWorld(N=5242, E=14484):
    """
    :param - N: number of nodes
    :param - E: number of edges

    return type: snap.PUNGraph
    return: Small-World graph with N nodes and E edges
    """
    Graph = genCircle(N)
    Graph = connectNbrOfNbr(Graph, N)
    Graph = connectRandomNodes(Graph, 4000)
    return Graph

smallworld_graph = genSmallWorld(N=5242, E=14484)
assert smallworld_graph.GetEdges() == 14484
assert smallworld_graph.GetNodes() == 5242

In [ ]:
# !wget http://snap.stanford.edu/data/ca-GrQc.txt.gz
# !gunzip ca-GrQc.txt.gz  
!ls

In [ ]:
!head -n 7 ca-GrQc.txt

In [ ]:
def loadCollabNet(path):
    """
    :param - path: path to edge list file

    return type: snap.PUNGraph
    return: Graph loaded from edge list at `path and self edges removed

    Do not forget to remove the self edges!
    """
    ############################################################################
    df = pd.read_csv('ca-GrQc.txt',skiprows=4,sep='\t', header=None)
    df.columns = ['src','dst']
    df['src'] = df['src'].astype(int)
    df['dst'] = df['dst'].astype(int)
    rw_graph = snap.TUNGraph.New()
    for idx, row in enumerate(df.itertuples()):
        if not rw_graph.IsNode(row.src):
            rw_graph.AddNode(row.src)
        if not rw_graph.IsNode(row.dst):
            rw_graph.AddNode(row.dst)
        
        if row.src == row.dst:
            continue
        rw_graph.AddEdge(row.src,row.dst)

    ############################################################################
    return rw_graph

collab_graph = loadCollabNet('ca-GrQc.txt')

assert collab_graph.GetNodes() == 5242
assert collab_graph.GetEdges() == 14484

In [ ]:
def getDataPointsToPlot(Graph):
    """
    :param - Graph: snap.PUNGraph object representing an undirected graph

    return values:
    X: list of degrees
    Y: list of frequencies: Y[i] = fraction of nodes with degree X[i]
    """
    ############################################################################
    degs = [node.GetOutDeg() for node in Graph.Nodes()]
    df = pd.Series(degs,name='deg').value_counts().reset_index().rename(columns={'index':'deg','deg':'count'})
    df['count_ratio'] = df['count']/df['count'].sum()
    df = df.sort_values('deg',ascending=True)
    
    
    X, Y = df['deg'],df['count_ratio']

    ############################################################################
    return X, Y


In [ ]:
erdosRenyi = None
smallWorld = None
collabNet = None

def Q1_1():
    """
    Code for HW1 Q1.1
    """
    global erdosRenyi, smallWorld, collabNet
    erdosRenyi = genErdosRenyi(5242, 14484)
    smallWorld = genSmallWorld(5242, 14484)
    collabNet = loadCollabNet("ca-GrQc.txt")

    x_erdosRenyi, y_erdosRenyi = getDataPointsToPlot(erdosRenyi)
    plt.loglog(x_erdosRenyi, y_erdosRenyi, color = 'y', label = 'Erdos Renyi Network')

    x_smallWorld, y_smallWorld = getDataPointsToPlot(smallWorld)
    plt.loglog(x_smallWorld, y_smallWorld, linestyle = 'dashed', color = 'r', label = 'Small World Network')

    x_collabNet, y_collabNet = getDataPointsToPlot(collabNet)
    plt.loglog(x_collabNet, y_collabNet, linestyle = 'dotted', color = 'b', label = 'Collaboration Network')

    plt.xlabel('Node Degree (log)')
    plt.ylabel('Proportion of Nodes with a Given Degree (log)')
    plt.title('Degree Distribution of Erdos Renyi, Small World, and Collaboration Networks')
    plt.legend()
    plt.show()

Q1_1()

### 1.2 Clustering Coefficient

In [5]:
def getNbrEdgeCount(Node, Graph):
    """
    Get the number of edges among neighbors of a node
    """
    nbrs = [e for e in Node.GetOutEdges()]

    nbr_edge_count = 0
    for i in nbrs:
        curr_nbr = Graph.GetNI(i)
        # loop through other edges and count number of edges
        for j in nbrs:
            if j >= i:
                break
            if curr_nbr.IsNbrNId(j):
                nbr_edge_count += 1
                
    return nbr_edge_count

In [ ]:
def calcClusteringCoefficientSingleNode(Node, Graph):
    """
    :param - Node: node from snap.PUNGraph object. Graph.Nodes() will give an
                   iterable of nodes in a graph
    :param - Graph: snap.PUNGraph object representing an undirected graph

    return type: float
    returns: local clustering coeffient of Node
    """
    ############################################################################
    if Node.GetOutDeg() < 2:
        return 0

    nbr_edge_count = getNbrEdgeCount(Node, Graph)
    num_nbrs = len([e for e in Node.GetOutEdges()])            
    max_edge_count = (num_nbrs * (num_nbrs - 1))/2
    C = nbr_edge_count/max_edge_count
    ############################################################################
    return C

def calcClusteringCoefficient(Graph):
    """
    :param - Graph: snap.PUNGraph object representing an undirected graph

    return type: float
    returns: clustering coeffient of Graph
    """
    ############################################################################
    # TODO: Your code here! If you filled out calcClusteringCoefficientSingleNode,
    #       you'll probably want to call it in a loop here
    c_sum = 0
    for node in Graph.Nodes():
        c_sum += calcClusteringCoefficientSingleNode(node, Graph)
        
    C = c_sum/Graph.GetNodes()

    ############################################################################
    return C

def Q1_2():
    """
    Code for Q1.2
    """
    C_erdosRenyi = calcClusteringCoefficient(erdosRenyi)
    C_smallWorld = calcClusteringCoefficient(smallWorld)
    C_collabNet = calcClusteringCoefficient(collabNet)

    print('Clustering Coefficient for Erdos Renyi Network: %f' % C_erdosRenyi)
    print('Clustering Coefficient for Small World Network: %f' % C_smallWorld)
    print('Clustering Coefficient for Collaboration Network: %f' % C_collabNet)
    
Q1_2()

In [ ]:
print('SNAP Clustering Coefficient for Erdos Renyi Network: %f' % snap.GetClustCf(erdosRenyi))
print('SNAP Clustering Coefficient for Small World Network: %f' % snap.GetClustCf(smallWorld))
print('SNAP Clustering Coefficient for Collaboration Network: %f' % snap.GetClustCf(collabNet))

## 2 Structural Roles: Rolx and ReFex

In [2]:
FIn = snap.TFIn('hw1-q2.graph')
Graph = snap.TUNGraph.Load(FIn)

### 2.1 Basic Features

In [16]:
def get_egonet_connections(node, Graph):
    """
    edges in an out of egonet
    """
    nbrs = [e for e in node.GetOutEdges()]
    egonet_nodes = nbrs + [node.GetId()]
    logging.debug(f'\n\ncurrent node - {node.GetId()}')
    logging.debug(f'egonet nodes:  {egonet_nodes}')
    num_ego_out_edges = 0
    for nbr in nbrs:
        nbr_edges = [e for e in Graph.GetNI(nbr).GetOutEdges()]
        nbr_out_nodes = [o for o in nbr_edges if o not in egonet_nodes]
        logging.debug(f'neighbor: {nbr} neighbor edges: {nbr_edges}. nbr-out: {nbr_out_nodes}')
        num_ego_out_edges += len(nbr_out_nodes)
    logging.debug(f'num of ego out edges: {num_ego_out_edges}')
    return num_ego_out_edges

In [211]:
# %%time
logger.setLevel(logging.INFO)
node_degrees = []
nodes = []
num_ego_edges = []
num_ego_connections = []
for Node in Graph.Nodes():
    logging.debug(Node.GetId())
#     if Node.GetId() > 10:
#         break
    nodes.append(Node.GetId())
    node_degrees.append(Node.GetOutDeg())
    
    # edges among neighbors
    num_ego_edges.append(getNbrEdgeCount(Node, Graph) + Node.GetOutDeg())
    
    # get egonet edge  
    num_ego_connections.append(get_egonet_connections(Node, Graph))
    
df = pd.DataFrame({
    'node': nodes,
    'degree': node_degrees,
    'num_ego_edges': num_ego_edges,
    'ego_cons': num_ego_connections
})
print(df.shape)
print(df.loc[9])

(1589, 4)
node              9
degree            6
num_ego_edges    10
ego_cons          1
Name: 9, dtype: int64


report the top 5 nodes that are most similar to node 9

In [229]:
def cosine_similarity_with(row, with_idx, df,cols):
    x = row[cols].values
    y = df.loc[with_idx][cols].values
    
    if np.sum(np.square(x)) == 0 or np.sum(np.square(x)) == 0 or row.node == with_idx:
        sim = 0
    else:
        sim = x.dot(y)/(np.sqrt(x.dot(x)) * np.sqrt(y.dot(y)))
    return sim

feature_cols = ['degree','num_ego_edges','ego_cons']
df['sim'] = df.apply(cosine_similarity_with, axis=1, with_idx=9, df=df.copy(), cols=feature_cols)
df.sort_values('sim',inplace=True, ascending=False)
df.head()

,node,degree,num_ego_edges,ego_cons,sim,nbr_mean_degree,nbr_mean_num_ego_edges,nbr_mean_ego_cons,nbr_sum_degree,nbr_sum_num_ego_edges,nbr_sum_ego_cons
415,415,7,11,1,0.999616,5.857143,5.857143,5.857143,41.0,41.0,41.0
286,286,3,5,0,0.996344,1.333333,1.333333,1.333333,4.0,4.0,4.0
288,288,3,5,0,0.996344,1.333333,1.333333,1.333333,4.0,4.0,4.0
1054,1054,4,7,0,0.996118,1.500000,1.500000,1.500000,6.0,6.0,6.0
1336,1336,4,7,0,0.996118,1.500000,1.500000,1.500000,6.0,6.0,6.0


### 2.2 Recursive Features

In [219]:
# create the new columns
nbr_mean_cols = [f'nbr_mean_{col}' for col in feature_cols]
nbr_sum_cols = [f'nbr_sum_{col}' for col in feature_cols]
for col in (nbr_mean_cols + nbr_sum_cols):
    df[col] = 0.0
    
for Node in Graph.Nodes():
    # get the neighbor node ids for each node
    nbr_node_ids = [Node.GetNbrNId(idx) for idx in range(Node.GetOutDeg())]
    if not nbr_node_ids:
        continue

    # calculate means of neighbor features and set value at node for each column
    nbr_means = df.loc[nbr_node_ids][feature_cols].mean(axis=0)
    for col_idx in range(len(feature_cols)):
        df.at[Node.GetId(), nbr_mean_cols[col_idx]] = nbr_means[feature_cols[idx]]

    # calculate sums of neighbor features and set value at node for each column
    nbr_sums = df.loc[nbr_node_ids][feature_cols].sum(axis=0)
    for col_idx in range(len(feature_cols)):
        df.at[Node.GetId(), nbr_sum_cols[col_idx]] = nbr_sums[feature_cols[idx]]

In [232]:
simility_cols = feature_cols + nbr_mean_cols + nbr_sum_cols
df['sim'] = df.apply(cosine_similarity_with, axis=1, with_idx=9, df=df.copy(), cols=simility_cols)
df.sort_values('sim',inplace=True, ascending=False)
df.head()

,node,degree,num_ego_edges,ego_cons,sim,nbr_mean_degree,nbr_mean_num_ego_edges,nbr_mean_ego_cons,nbr_sum_degree,nbr_sum_num_ego_edges,nbr_sum_ego_cons
496,496,6,9,0,0.999488,4.000000,4.000000,4.000000,24.0,24.0,24.0
537,537,7,14,0,0.999273,4.571429,4.571429,4.571429,32.0,32.0,32.0
320,320,6,13,0,0.999214,4.666667,4.666667,4.666667,28.0,28.0,28.0
24,24,5,10,2,0.998898,4.400000,4.400000,4.400000,22.0,22.0,22.0
25,25,5,10,2,0.998898,4.400000,4.400000,4.400000,22.0,22.0,22.0
